In [210]:
#передискретизация сигнала 
import plotly as py
py.offline.init_notebook_mode(connected=True)
import numpy as np
from sklearn.metrics import mean_squared_error


import math
 
# Recursive function to return gcd 
# of a and b
def gcd(a,b) :
    if (a < b) :
        return gcd(b, a)
     
    # base case
    if (abs(b) < 0.001) :
        return a
    else :
        return (gcd(b, a - math.floor(a / b) * b))

def lcm_f(a, b):
    return a * b // gcd(a, b)


def lcm(a,b):
    m = a*b
    while a != 0 and b != 0:
        if a > b:
            a %= b
        else:
            b %= a
    return m // (a+b)

def create_notch_filter(kernel_size):
    if kernel_size % 2 == 0:
        raise ValueError("kernel_size should be odd")
    return [1 / kernel_size for i in range(kernel_size)]
    
def notch_filter(signal, filt):
    return np.convolve(signal, filt, mode='valid');

def impulse_response_notch(filt):
    f = np.pad(filt, (0, len(filt) * 4), 'constant', constant_values=(0,0))
    return np.fft.fft(f)[:int(len(f) / 2)]

#batterworth
def prod(iterable):
    from functools import reduce
    from operator import mul
    return reduce(mul, iterable, 1)

def transfer_function_butterworth(Rp, Rs, w0, w1):
    ep = np.sqrt(np.power(10, Rp / 10.) - 1)
    es = np.sqrt(np.power(10, Rs / 10.) - 1)
    k = w0 / w1
    k1 = ep / es
    N = int(np.ceil(np.log(k1) / np.log(k)))
    L = N // 2
    r = N % 2
    a = np.power(ep, - 1. / N)
    th = [(2 * n + 1) / 2 / N * np.pi for n in range(L)]
    
    return lambda s: 1. / (ep * np.power(s + a, r) * prod(s*s + 2*a*np.sin(thn)*s + a*a for thn in th))

def interpolate(s1, f1, f2):
    if(f1 >= f2):
        return s1
    step = int(f2/f1 + 0.5)   
    s = np.zeros(N*(step-1))
    print("interpolate length ", len(s))
    for l in range(N - 1):
     s[l*(step-1)] = s1[l] 
    Rp = 1.0
    Rs = 30.0
    w0 = 17.0
    w1 = 25.0
    HGen = transfer_function_butterworth(Rp, Rs, w0, w1)
    df = 1. / T
    k = [i * df for i in range(len(s))]
    H_jw = [HGen(1j*i / w0) for i in k]
    afr = np.absolute(H_jw)
    coef = np.fft.ifft(afr)
    coef[50:-50] = 0
    H_jw = np.fft.fft(coef.real)
    afr = np.absolute(H_jw)
    afr = afr[:len(afr) // 2]
    filtsym = np.append(afr, afr[::-1])   
    discr = np.fft.ifft(np.fft.fft(s) * filtsym).real * step * round(np.sqrt(f2/f1 - 0.96) , 1)
    
    return discr

def decimate(s1, f1, f2):
    if(f1 == f2):
        return s1
    Rp = 1.0
    Rs = 30.0
    w0 = 20
    w1 = 25 #max(f1, f2) #25 
    HGen = transfer_function_butterworth(Rp, Rs, w0, w1)
    df = 1. / T
    k = [i * df for i in range(len(s1))]
    H_jw = [HGen(1j*i / w0) for i in k]
    afr = np.absolute(H_jw)
    coef = np.fft.ifft(afr)
    coef[50:-50] = 0
    H_jw = np.fft.fft(coef.real)
    afr = np.absolute(H_jw)
    afr = afr[:len(afr) // 2]
    filtsym = np.append(afr, afr[::-1])    
    discr1 = np.fft.ifft(np.fft.fft(s1) * filtsym).real
    step = int(f1/f2 + 0.5) 
    s = np.zeros(N//step) 
    print("decimate length ", len(s))
    for l in range(len(s)-1):
        s[l] = s1[l*(step)] 
    return s

In [220]:
from plotly.graph_objs import Scatter, Layout
N = 1024
T = 2
t = np.linspace(0, T, N)
f1 = 1
f2 = 4

f3 = 8

s1 = [np.sin(np.pi * 2 *f1 * i) + np.sin(np.pi * 2 *f2 * i) for i in t]
s2 = [np.sin(np.pi * 2 *f2 * i) for i in t]

print("signal 1 lengths")
print("oringinal length", len(s1))

f1 = len(s1)
f3 = f1 * 2.5

ftmp = lcm_f(f1, f3)
print(ftmp)
stmp = interpolate(s1, f1, ftmp)
discr1 = decimate(stmp, ftmp, f3)

print((f3))
    
t1 = np.linspace(0, T, len(discr1))

'''
print("signal 2 lengths")
print("oringinal length", len(s2))

ftmp = lcm(f2, f3)
discr2 = stmp = interpolate(s2, f2, ftmp)
#discr2 = decimate(stmp, ftmp, f3)
    

t2 = np.linspace(0, T, len(discr2))


tr_s2 = Scatter(
    x = t,
    y = s2,
    name='signal2'
)

tr_diskr2 = Scatter(
    x = t2,
    y = discr2,
    name='discr2'
)

'''
tr_s1 = Scatter(
    x = t,
    y = s1,
    name='signal1'
)



tr_diskr1 = Scatter(
    x = t1,
    y = discr1,
    name='discr1'
)



fig = py.tools.make_subplots(rows = 1, cols = 1)
fig['layout'].update(height=450, width=900)
fig.append_trace(tr_s1, 1, 1)
fig.append_trace(tr_diskr1, 1, 1)
#fig.append_trace(tr_s2, 1, 1)
#fig.append_trace(tr_diskr2, 1, 1)
py.offline.iplot(fig)

signal 1 lengths
oringinal length 1024
5120.0
interpolate length  4096
decimate length  512
2560.0
This is the format of your plot grid:
[ (1,1) x1,y1 ]



In [218]:
N = 1024
T = 2
t = np.linspace(0, T, N)
f1 = 15
f2 = 8

f3 = 15

s1 = [np.sin(np.pi * 2 *f1 * i) for i in t]
s2 = [np.sin(np.pi * 2 *f2 * i) for i in t]

print("signal 1 lengths")
print("oringinal length", len(s1))
if f3 > f1:
    discr1 = interpolate(s1, f1, f3)
else:
    discr1 = decimate(s1, f1, f3)
t1 = np.linspace(0, T, len(discr1))

print("signal 2 lengths")
print("oringinal length", len(s2))
if f3 > f2:
    discr2 = interpolate(s2, f2, f3)
else:
    discr2 = decimate(s2, f2, f3)
t2 = np.linspace(0, T, len(discr2))

tr_s1 = Scatter(
    x = t,
    y = s1,
    name='signal1'
)

tr_s2 = Scatter(
    x = t,
    y = s2,
    name='signal2'
)

tr_diskr1 = Scatter(
    x = t1,
    y = discr1,
    name='discr1'
)

tr_diskr2 = Scatter(
    x = t2,
    y = discr2,
    name='discr2'
)

fig = py.tools.make_subplots(rows = 2, cols = 1)
fig['layout'].update(height=450, width=900)
fig.append_trace(tr_s1, 1, 1)
fig.append_trace(tr_diskr1, 1, 1)
fig.append_trace(tr_s2, 2, 1)
fig.append_trace(tr_diskr2, 2, 1)
py.offline.iplot(fig)

signal 1 lengths
oringinal length 1024
signal 2 lengths
oringinal length 1024
interpolate length  1024
This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]

